<a href="https://colab.research.google.com/github/Sharath-11507563/Aspect-based-sentimental-analysis-on-Rulemaking-comments/blob/main/data_scrap_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
key = "5oXgUp9JOCQI4zE4gEennlnQpQQzEk2OqXdq6qCW"
from bs4 import BeautifulSoup
import requests, json
import pandas as pd

In [ ]:
url = "https://api.regulations.gov/v4/dockets?filter[searchTerm]=Tobacco Products; Required Warnings for Cigarette Packages and Advertisements&api_key=" + key # creating the url
soup = BeautifulSoup(requests.get(url).content, 'lxml')# Request for data
# Extracting the doket id and object ids
data = json.loads(soup.body.p.text)["data"]  
docket_id = [i['id'] for i in data]
docket_object_id = [i['attributes']['objectId'] for i in data]

In [ ]:
# This cell will go through all the dockets and get their documents ids and object ids
document_ids = []
document_objectId = []
for dockt in docket_id:
    url = "https://api.regulations.gov/v4/documents?filter[docketId]="+dockt+"&api_key=" + key
    soup = BeautifulSoup(requests.get(url).content, 'lxml')
    data = json.loads(soup.body.p.text)["data"]
    document_ids.append([i['id'] for i in data]) # get document ids
    document_objectId.append([i['attributes']['objectId'] for i in data]) # get document object ids 
document_ids = list(set([j for i in document_ids for j in i])) # removing similar ids 
document_objectId = list(set([j for i in document_objectId for j in i])) # Removing similar object ids of documents

In [ ]:
# This cell will get all the comments ids from the above scrapped document object ids
commentsIds = []
for doc_objectId in document_objectId:
    #print(doc_objectId)
    for num in range(1, 21): # check for first 5000 comments from each document
        url = "https://api.regulations.gov/v4/comments?filter[commentOnId]="+doc_objectId+"&page[size]=250&page[number]="+str(num)+"&sort=lastModifiedDate,documentId&api_key="+ key
        soup = BeautifulSoup(requests.get(url).content, 'lxml')
        try:
            comment_id_data = json.loads(soup.body.p.text)['data'] 
            if comment_id_data==[]: break # break the loop if their is seq of empty pages
            commentsIds.append([i['id'] for i in comment_id_data]) 
        except: break
commentsIds = list(set([j for i in commentsIds for j in i if j!=[]])) # remove similar comment ids

In [ ]:
# this cell will get comments from each comment id and store it into dict formate
comments = {}
for comment_id in commentsIds:
    url = "https://api.regulations.gov/v4/comments/"+comment_id+"?include=attachments&api_key="+ key
    soup = BeautifulSoup(requests.get(url).content, 'lxml')
    try: comments[comment_id]= json.loads(soup.body.p.text)["data"]["attributes"]["comment"]
    except: pass

In [ ]:
# Save the comments
df = pd.DataFrame(list(comments.items()), columns=["commentIds", 'comments'])
df.to_csv('Comments6.csv', index = False)
from google.colab import files
files.download("Comments6.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>